In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import datetime

In [3]:
# Read the data file
df=pd.read_csv('LoanStats3b_securev1.csv',header=1)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,49,129,130,131,134,135,136,139) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# Choose some meaningful and available features for predictions
df=df[['loan_amnt','term','int_rate','installment','sub_grade','emp_title','emp_length','home_ownership','annual_inc','verification_status',
        'issue_d','desc','purpose','title','zip_code','addr_state','dti','delinq_2yrs','earliest_cr_line','fico_range_low','fico_range_high','inq_last_6mths',
        'open_acc','revol_bal','revol_util','total_acc','acc_open_past_24mths','mort_acc','percent_bc_gt_75','loan_status']]


In [11]:
# Drop rows that are completely empty
df.dropna(how='all', inplace=True)

In [12]:
# Change the format of int_rate to float
df['int_rate']=df['int_rate'].apply(lambda x: float(x[:-1]))

In [13]:
# Change the format of revol_util to float
def process_revol_util(x):
    if type(x) == str:
        x=float(x[:-1])
    return x
df['revol_util']=df['revol_util'].apply(lambda x: process_revol_util(x))


In [14]:
# Fill in missing values with the mean
df['revol_util'].fillna(value=df['revol_util'].mean(), inplace=True)
df['acc_open_past_24mths'].fillna(value=df['acc_open_past_24mths'].mean(), inplace=True)
df['mort_acc'].fillna(value=df['mort_acc'].mean(), inplace=True)
df['percent_bc_gt_75'].fillna(value=df['percent_bc_gt_75'].mean(), inplace=True)

In [15]:
# Change emp_length and sub_grade from strings to integers
mapping_dict={"emp_length": {"10+ years": 10, "9 years": 9, "8 years": 8, "7 years": 7, "6 years": 6, "5 years": 5, "4 years": 4, 
                               "3 years": 3, "2 years": 2, "1 year": 1, "< 1 year": 0}, 
                "sub_grade":{"G5": 0, "G4": 1, "G3": 2, "G2": 3, "G1": 4, "F5": 5, "F4": 6, "F3": 7, "F2": 8, "F1": 9, "E5": 10,
                             "E4": 11, "E3": 12, "E2": 13, "E1": 14, "D5": 15, "D4": 16, "D3": 17, "D2": 18, "D1": 19, "C5": 20, 
                             "C4": 21, "C3": 22, "C2": 23, "C1": 24, "B5": 25, "B4": 26, "B3": 27, "B2": 28, "B1": 29, "A5": 30, 
                             "A4": 31, "A3": 32, "A2": 33, "A1": 34}}
df=df.replace(mapping_dict)


In [17]:
# Fill in missing values of emp_length based on emp_title
for i in df.index:
    if np.isnan(df.loc[i,'emp_length']):
        title=df.loc[i,'emp_title']
        if type(title)==str:
            average=df[df['emp_title']==title]['emp_length'].mean()
            df.loc[i,'emp_length']=average

In [23]:
# Drop the rows with NaN in either emp_title or emp_length
df.dropna(subset=['emp_length'], inplace=True)

In [24]:
# Change issue_d to datetime format
df['issue_d']=df['issue_d'].apply(lambda x: datetime.datetime.strptime(x, '%b-%Y'))

In [25]:
# Change earliest_cr_line to datetime format
df['earliest_cr_line']=df['earliest_cr_line'].apply(lambda x: datetime.datetime.strptime(x, '%b-%Y'))

In [26]:
# Calculate the differences in months from earliest_cr_line to issue_d
def diff_month(d1,d2):
    return (d1.year-d2.year)*12+(d1.month-d2.month)

for i in df.index:
    df.loc[i,'earliest_cr_line']=diff_month(df.loc[i,'issue_d'],df.loc[i,'earliest_cr_line'])

In [27]:
# Creat dummy variables for the rest categorical variables
dummy_df=pd.get_dummies(df[['term','home_ownership','verification_status','purpose','addr_state']])
df=pd.concat([df, dummy_df], axis=1)
df=df.drop(['term','home_ownership','verification_status','purpose','addr_state'], axis=1)

In [28]:
# Change the target variable to numbers: 1 indicates defaulted, 0 indicates paid off
def flag(x):
    if x=='Fully Paid':
        return 0
    elif x=='Charged Off':
        return 1

df['loan_status']=df['loan_status'].apply(flag)

# Drop loans with other unknown status
df.dropna(subset=['loan_status'], inplace=True)

In [29]:
# Reset the index to integers starting from 0
df.reset_index(drop=True, inplace=True)

In [30]:
df

,loan_amnt,int_rate,installment,sub_grade,emp_title,emp_length,annual_inc,issue_d,desc,title,...,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY
0,12000.0,7.62,373.94,32,Systems Engineer,3.0,96500.0,2013-12-01,Borrower added on 12/31/13 > Bought a new ho...,Debt Consolidation and Credit Transfer,...,0,0,1,0,0,0,0,0,0,0
1,27050.0,10.99,885.46,28,Team Leadern Customer Ops & Systems,10.0,55000.0,2013-12-01,Borrower added on 12/31/13 > Combining high ...,Debt Consolidation,...,0,0,0,0,0,0,0,0,0,0
2,12000.0,11.99,398.52,27,LTC,10.0,130000.0,2013-12-01,NaN,Debt consolidation,...,0,0,0,0,0,0,0,0,0,0
3,28000.0,7.62,872.52,32,Area Sales Manager,5.0,325000.0,2013-12-01,NaN,Pay off other Installment loan,...,0,0,0,0,0,0,0,0,0,0
4,27600.0,19.97,730.78,15,Street Operations Supervisor,6.0,73000.0,2013-12-01,Borrower added on 12/31/13 > I had some wate...,Consolidation of debt and home improve.,...,0,0,0,0,0,0,0,0,0,0
5,12000.0,10.99,392.81,28,Project Manager,4.0,60000.0,2013-12-01,Borrower added on 12/31/13 > I would like to...,No Regrets,...,0,0,0,0,0,0,0,0,0,0
6,11100.0,14.98,384.68,22,Teacher,10.0,90000.0,2013-12-01,NaN,Other,...,0,0,0,0,0,0,0,0,0,0
7,12000.0,13.53,407.40,25,On road manager,10.0,40000.0,2013-12-01,NaN,Debt consolidation,...,0,0,0,0,0,0,0,0,0,0
8,9750.0,13.98,333.14,24,Medical Assistant,1.0,26000.0,2013-12-01,Borrower added on 12/31/13 > While being in ...,Debt Consilation,...,0,0,0,0,0,0,0,0,0,0
9,4800.0,10.99,157.13,28,Surgical Technician,2.0,39600.0,2013-12-01,Borrower added on 12/31/13 > Just bought a h...,For The House,...,0,0,1,0,0,0,0,0,0,0


In [31]:
df.dtypes

loan_amnt                         float64
int_rate                          float64
installment                       float64
sub_grade                           int64
emp_title                          object
emp_length                        float64
annual_inc                        float64
issue_d                    datetime64[ns]
desc                               object
title                              object
zip_code                           object
dti                               float64
delinq_2yrs                       float64
earliest_cr_line                    int64
fico_range_low                    float64
fico_range_high                   float64
inq_last_6mths                    float64
open_acc                          float64
revol_bal                         float64
revol_util                        float64
total_acc                         float64
acc_open_past_24mths              float64
mort_acc                          float64
percent_bc_gt_75                  

In [32]:
df.isnull().sum()

loan_amnt                       0
int_rate                        0
installment                     0
sub_grade                       0
emp_title                    3895
emp_length                      0
annual_inc                      0
issue_d                         0
desc                       102308
title                           6
zip_code                        0
dti                             0
delinq_2yrs                     0
earliest_cr_line                0
fico_range_low                  0
fico_range_high                 0
inq_last_6mths                  0
open_acc                        0
revol_bal                       0
revol_util                      0
total_acc                       0
acc_open_past_24mths            0
mort_acc                        0
percent_bc_gt_75                0
loan_status                     0
term_ 36 months                 0
term_ 60 months                 0
home_ownership_MORTGAGE         0
home_ownership_NONE             0
home_ownership

In [33]:
df.describe()

,loan_amnt,int_rate,installment,sub_grade,emp_length,annual_inc,dti,delinq_2yrs,earliest_cr_line,fico_range_low,...,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY
count,180309.000000,180309.000000,180309.000000,180309.000000,180309.000000,1.803090e+05,180309.000000,180309.000000,180309.000000,180309.000000,...,180309.000000,180309.000000,180309.000000,180309.000000,180309.000000,180309.000000,180309.000000,180309.000000,180309.000000,180309.000000
mean,14476.932654,14.286212,447.134435,23.247897,6.090532,7.323390e+04,17.012586,0.241480,184.132107,696.823453,...,0.002180,0.010709,0.078105,0.008008,0.031074,0.001608,0.023948,0.012240,0.004692,0.002451
std,8121.452157,4.442508,242.805641,6.419024,3.565751,5.237876e+04,7.578693,0.706113,82.139195,29.886771,...,0.046635,0.102931,0.268337,0.089131,0.173519,0.040072,0.152887,0.109956,0.068337,0.049450
min,1000.000000,6.000000,4.930000,0.000000,0.000000,5.000000e+03,0.000000,0.000000,36.000000,660.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8000.000000,11.140000,273.920000,19.000000,3.000000,4.500000e+04,11.300000,0.000000,129.000000,675.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,12500.000000,14.090000,399.970000,24.000000,6.000000,6.305940e+04,16.730000,0.000000,169.000000,690.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,20000.000000,17.270000,581.180000,28.000000,10.000000,8.803300e+04,22.500000,0.000000,225.000000,710.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,35000.000000,26.060000,1408.130000,34.000000,10.000000,7.141778e+06,34.990000,29.000000,750.000000,845.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [34]:
df.to_csv('2012-2013_cleaned.csv')

In [3]:
df=pd.read_csv('2012-2013_cleaned.csv', index_col=0)

In [8]:
corrdf=df.iloc[:,:22].corr()

In [10]:
corrdf[corrdf>0.4]

,loan_amnt,int_rate,installment,sub_grade,emp_length,annual_inc,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,open_acc,revol_bal,revol_util,total_acc,acc_open_past_24mths,mort_acc,percent_bc_gt_75
loan_amnt,1.00000,NaN,0.95415,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
int_rate,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
installment,0.95415,NaN,1.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub_grade,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,0.519093,0.519089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
emp_length,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
annual_inc,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
dti,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
delinq_2yrs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
earliest_cr_line,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fico_range_low,NaN,NaN,NaN,0.519093,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
